# On utilisera le truc etude n° 10

In [148]:
import numpy as np
import msicpe.san as san
from plotly import express as px
import scipy.signal as signal


H= 5
for lettre in ['a', 'e', 'i', 'o', 'u']:
    signaux = []
    for i in range(5):
        dat = np.load(f'data_partie_2/apprentissage/{lettre}_{i+1}.npz') # dat est un dictionnaire contenant les données contenues dans le fichier .npz
        s = dat['s'] # extraction du signal utile
        t = dat['t'] # extraction du vecteur temps associé au signal

        nu, S = san.trans_fourier(s, t)
        S = np.abs(S)

        s_centre= s - np.mean(s)
        nu, S_norm = san.trans_fourier(s_centre, t)
        S_norm = np.abs(S_norm)
        S_norm /= np.max(S_norm)

        f0 = san.detect_fondamentale(S_norm, nu)

        # Liste des harmoniques
        freq_list = [f0 * k for k in range(1, H+1)]

        freq_peaks, amp_peaks = san.detect_pics(S, freq_list, nu)

        vec = np.zeros(H+1)
        vec[0] = np.log10(np.max(S))
        vec[1:] = np.log10(amp_peaks)
        signaux.append(vec)

    k = np.arange(H+1)
    vec_moy = np.mean(signaux, axis=0)

    k = list(range(H+1))
    indice = []
    ampl = []
    labels = []

    for i in range(5):
        ampl.extend(k)
        indice.extend(signaux[i])
        labels.extend([f"Son {i+1}"] * (H+1))

    ampl.extend(k)
    indice.extend(vec_moy)
    labels.extend(["Moyenne"] * (H+1))

    fig = px.line(x=ampl, y=indice, color=labels, markers=True,title=f"Voyelle: {lettre} - Spectres (log10) des 5 sons + vecteur moyen", labels={"x": "Indice d'harmonique k", "y": "Amplitude log10"}).show()

